# Mohamed diop Master 2 BI&BIG DATA 

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.9` // Not required since almond 0.7.0 (will be automatically added when importing spark)
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SparkSession
import org.apache.spark.storage.StorageLevel
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._
import $ivy.`org.apache.spark::spark-mllib:2.4.5`


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                // Not required since almond 0.7.0 (will be automatically added when importing spark)

import org.apache.spark.sql.functions._

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.SparkSession

import org.apache.spark.storage.StorageLevel

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import $ivy.$                                    


In [2]:
import org.apache.log4j.{Level, Logger}

val rootLogger = Logger.getRootLogger()
rootLogger.setLevel(Level.ERROR)

Logger.getLogger("org.apache.spark").setLevel(Level.WARN)
Logger.getLogger("org.spark-project").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


rootLogger: Logger = org.apache.log4j.spi.RootLogger@611054f8

In [3]:
import org.apache.spark.sql._

val spark = {
    SparkSession.builder()
    .master("local[*]")
    .appName("Detection_Fraude_Master2")
    .config("spark.sql.shuffle.partitions", 6)
    .getOrCreate()

}

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
23/03/25 22:15:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/25 22:15:07 ERROR Shell: Failed to locate the winutils binary in the hadoop binary path
java.io.IOException: Could not locate executable null\bin\winutils.exe in the Hadoop binaries.
	at org.apache.hadoop.util.Shell.getQualifiedBinPath(Shell.java:378)
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:393)
	at org.apache.hadoop.util.Shell.<clinit>(Shell.java:386)
	at org.apache.hadoop.util.StringUtils.<clinit>(StringUtils.java:79)
	at org.apache.hadoop.security.Groups.parseStaticMapping(Groups.java:116)
	at org.apache.hadoop.security.Groups.<init>(Groups.java:93)
	at org.apache.hadoop.security.Groups.<init>(Groups.java:73)
	at org.apache.hadoop.security.Groups.getUserToGroupsMappingService(Groups.java:293)
	at org.apache.hadoop.security.UserGroupInform

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@7e81f7b1

# Chargement des données de transactions

In [4]:
val cb_fraude = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("creditcard.csv")

cb_fraude: DataFrame = [Time: decimal(10,0), V1: double ... 29 more fields]

# afficher le schéma du DataFrame

In [5]:
cb_fraude.printSchema()

root
 |-- Time: decimal(10,0) (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double

# Afficher les statistiques sommaires

In [6]:
cb_fraude.describe().show()

23/03/25 22:15:17 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


+-------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|summary|             Time|                  V1|                  V2|                  V3|                  V4|                  V5|                  V6|                  V7|                  V8|                  V9|                 V10|                 V11|                 V12|                 V13|                 V14|                 V15|  

# Calcul du ratio de fraude dans le jeu de données

In [7]:
// Calculer la proportion de transactions frauduleuses
val fraudCount = cb_fraude.filter(col("Class") === 1).count()
val totalCount = cb_fraude.count()
val fraudRatio = fraudCount.toDouble / totalCount.toDouble
println(s"Fraud ratio: $fraudRatio")

Fraud ratio: 0.001727485630620034


fraudCount: Long = 492L
totalCount: Long = 284807L
fraudRatio: Double = 0.001727485630620034

In [8]:
// Visualiser la distribution des valeurs pour la colonne Amount
import spark.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


import spark.implicits._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


# Analyse des montants de transaction

In [9]:
cb_fraude.select("Amount").describe().show()
cb_fraude.select("Amount").rdd.map(r => r(0).asInstanceOf[Double]).histogram(10)

+-------+------------------+
|summary|            Amount|
+-------+------------------+
|  count|            284807|
|   mean| 88.34961925093077|
| stddev|250.12010924018867|
|    min|               0.0|
|    max|          25691.16|
+-------+------------------+



res8_1: (Array[Double], Array[Long]) = (
  Array(
    0.0,
    2569.116,
    5138.232,
    7707.348,
    10276.464,
    12845.58,
    15414.696,
    17983.811999999998,
    20552.928,
    23122.044,
    25691.16
  ),
  Array(284395L, 360L, 36L, 10L, 2L, 1L, 0L, 2L, 0L, 1L)
)

# Analyse de la colonne "Time"

In [47]:
// Visualiser la distribution des valeurs pour la colonne Time
cb_fraude.select("Time").describe().show()
cb_fraude.select("Time").rdd.map(r => r(0).asInstanceOf[Double]).histogram(10)

+-------+-----------------+
|summary|             Time|
+-------+-----------------+
|  count|           284807|
|   mean|       94813.8596|
| stddev|47488.14595456596|
|    min|                0|
|    max|           172792|
+-------+-----------------+



23/03/25 23:06:21 ERROR Executor: Exception in task 8.0 in stage 3390.0 (TID 23601)
java.lang.ClassCastException
23/03/25 23:06:21 WARN TaskSetManager: Lost task 8.0 in stage 3390.0 (TID 23601, localhost, executor driver): java.lang.ClassCastException

23/03/25 23:06:21 ERROR TaskSetManager: Task 8 in stage 3390.0 failed 1 times; aborting job


: 

# Visualisation de la distribution des valeurs de Time et Amount

In [11]:
import spark.implicits._

cb_fraude.select("Amount").describe().show()
cb_fraude.select("Amount").rdd.map(r => r(0).asInstanceOf[Double]).histogram(10)

// Visualiser la distribution des valeurs pour la colonne Time
cb_fraude.select("Time").describe().show()
cb_fraude.select("Time").rdd.map(r => r(0).asInstanceOf[Double]).histogram(10)

+-------+------------------+
|summary|            Amount|
+-------+------------------+
|  count|            284807|
|   mean| 88.34961925093077|
| stddev|250.12010924018867|
|    min|               0.0|
|    max|          25691.16|
+-------+------------------+

+-------+-----------------+
|summary|             Time|
+-------+-----------------+
|  count|           284807|
|   mean|       94813.8596|
| stddev|47488.14595456596|
|    min|                0|
|    max|           172792|
+-------+-----------------+



23/03/25 22:16:48 ERROR Executor: Exception in task 2.0 in stage 21.0 (TID 166)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 5.0 in stage 21.0 (TID 169)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 6.0 in stage 21.0 (TID 170)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 10.0 in stage 21.0 (TID 174)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 0.0 in stage 21.0 (TID 164)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 1.0 in stage 21.0 (TID 165)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 8.0 in stage 21.0 (TID 172)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 3.0 in stage 21.0 (TID 167)
java.lang.ClassCastException
23/03/25 22:16:48 ERROR Executor: Exception in task 7.0 in stage 21.0 (TID 171)
java.lang.ClassCastException
23/03/25 22:16:48 

: 

# Suppression des colonnes "Time" et "Amount" du DataFrame

In [12]:
val cb_fraude_clean = cb_fraude.drop("Time", "Amount")


cb_fraude_clean: DataFrame = [V1: double, V2: double ... 27 more fields]

# Split des données

In [13]:
val Array(trainingData, testData) = cb_fraude_clean.randomSplit(Array(0.8, 0.2), seed = 12345)


trainingData: Dataset[Row] = [V1: double, V2: double ... 27 more fields]
testData: Dataset[Row] = [V1: double, V2: double ... 27 more fields]

# Sélection des fonctionnalités à utiliser pour la détection de la fraude

In [14]:
// Sélection des caractéristiques pertinentes pour la détection de la fraude
val selectedFeatures = cb_fraude_clean.columns.filter(_ != "Class")

selectedFeatures: Array[String] = Array(
  "V1",
  "V2",
  "V3",
  "V4",
  "V5",
  "V6",
  "V7",
  "V8",
  "V9",
  "V10",
  "V11",
  "V12",
  "V13",
  "V14",
  "V15",
  "V16",
  "V17",
  "V18",
  "V19",
  "V20",
  "V21",
  "V22",
  "V23",
  "V24",
  "V25",
  "V26",
  "V27",
  "V28"
)

# Assemblage de vecteurs de fonctionnalités

In [15]:
// Assemblage des caractéristiques en un vecteur de fonctionnalités
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler()
.setInputCols(selectedFeatures)
.setOutputCol("features")

val assembledTrainingData = assembler.transform(trainingData)
val assembledTestData = assembler.transform(testData)

import org.apache.spark.ml.feature.VectorAssembler


assembler: VectorAssembler = vecAssembler_3de6008fffda
assembledTrainingData: DataFrame = [V1: double, V2: double ... 28 more fields]
assembledTestData: DataFrame = [V1: double, V2: double ... 28 more fields]

In [16]:
// Affichage des premières lignes du DataFrame après assemblage des caractéristiques
assembledTrainingData.show(5)

+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------------------+----------------+-----------------+-----+--------------------+
|               V1|              V2|               V3|              V4|               V5|               V6|               V7|              V8|               V9|              V10|             V11|              V12|             V13|              V14|               V15|              V16|              V17|              V18|              V19|             V20|             V21|              V22|              V23|              V24|             V25|               V26|           

# Comparaison du nombre de transactions frauduleuses dans l'ensemble de données d'entraînement et de test

In [17]:
// Affichage du nombre de transactions frauduleuses dans les ensembles d'entraînement et de test
val fraudCountTraining = assembledTrainingData.filter(col("Class") === 1).count()
val fraudCountTest = assembledTestData.filter(col("Class") === 1).count()
println(s"Fraud count in training set: $fraudCountTraining")
println(s"Fraud count in test set: $fraudCountTest")

Fraud count in training set: 392
Fraud count in test set: 100


fraudCountTraining: Long = 392L
fraudCountTest: Long = 100L

# Définition du modèle de régression logistique et des paramètres


In [18]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.VectorAssembler

// Définir le modèle de régression logistique
val lr = new LogisticRegression()
.setLabelCol("Class")
.setFeaturesCol("features")
.setMaxIter(10)
.setRegParam(0.0)

import org.apache.spark.ml.classification.LogisticRegression

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

import org.apache.spark.ml.feature.VectorAssembler

// Définir le modèle de régression logistique

lr: LogisticRegression = logreg_349cfaa06043

# Training du modèle de détection de fraude

In [19]:
// Entraîner le modèle avec les données d'entraînement
val model = lr.fit(assembledTrainingData)

23/03/25 22:17:18 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/25 22:17:18 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/03/25 22:17:19 INFO LBFGS: Step Size: 33,69
23/03/25 22:17:19 INFO LBFGS: Val and Grad Norm: 0,00712340 (rel: 0,438) 0,00515942
23/03/25 22:17:19 INFO LBFGS: Step Size: 1,500
23/03/25 22:17:19 INFO LBFGS: Val and Grad Norm: 0,00609361 (rel: 0,145) 0,00442296
23/03/25 22:17:19 INFO StrongWolfeLineSearch: Line search t: 0.12429927106991723 fval: 0.0053917202786732165 rhs: 0.0060935342410552255 cdd: -0.004799561857795584
23/03/25 22:17:19 INFO LBFGS: Step Size: 0,1243
23/03/25 22:17:19 INFO LBFGS: Val and Grad Norm: 0,00539172 (rel: 0,115) 0,00328031
23/03/25 22:17:19 INFO StrongWolfeLineSearch: Line search t: 0.4804681567060298 fval: 0.0046823908398160845 rhs: 0.005391612030422301 cdd: -9.567224902773158E-5
23/03/25 22:17:19 INFO LBFGS: Step Size: 0,4805
23/03/25 22:17:19 INFO 

model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_349cfaa06043, numClasses = 2, numFeatures = 28

# Prédiction avec le modèle

In [20]:
// Faire des prédictions sur les données de test
val predictions = model.transform(assembledTestData)

predictions: DataFrame = [V1: double, V2: double ... 31 more fields]

# Calcul de la métrique "Area under ROC" pour le modèle de détection de fraude.

In [21]:
// Évaluer la performance du modèle à l'aide de l'évaluateur d'évaluation de classification binaire
val evaluator = new BinaryClassificationEvaluator()
.setLabelCol("Class")
.setRawPredictionCol("rawPrediction")
.setMetricName("areaUnderROC")
val areaUnderROC = evaluator.evaluate(predictions)
println(s"Area under ROC: $areaUnderROC")

Area under ROC: 0.9705417184464612


evaluator: BinaryClassificationEvaluator = binEval_75508a95c02f
areaUnderROC: Double = 0.9705417184464612

# Affichage des prédictions, la classe réelle et les features correspondantes

In [22]:
// Afficher les prédictions avec la vraie valeur de la classe
predictions.select("prediction", "Class", "features").show()


+----------+-----+--------------------+
|prediction|Class|            features|
+----------+-----+--------------------+
|       0.0|    0|[-28.344757250015...|
|       0.0|    1|[-27.143678422949...|
|       1.0|    0|[-26.619952145208...|
|       0.0|    0|[-23.066841657925...|
|       1.0|    1|[-22.561699259129...|
|       1.0|    1|[-21.885433905174...|
|       1.0|    1|[-15.819178720771...|
|       1.0|    0|[-15.239439628061...|
|       1.0|    1|[-14.724627011925...|
|       0.0|    0|[-14.138924695625...|
|       0.0|    0|[-13.583082802692...|
|       0.0|    0|[-12.504075025054...|
|       0.0|    0|[-12.229230892692...|
|       0.0|    0|[-12.216458689092...|
|       0.0|    0|[-12.156495081502...|
|       0.0|    0|[-11.840668989052...|
|       0.0|    0|[-11.781028386318...|
|       0.0|    0|[-11.429498678806...|
|       0.0|    0|[-11.117454709763...|
|       0.0|    0|[-10.648382319781...|
+----------+-----+--------------------+
only showing top 20 rows



# Classification des transactions frauduleuses à l'aide de la régression logistique

In [23]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val lrModel = lr.fit(assembledTrainingData)

val predictions = lrModel.transform(assembledTestData)

23/03/25 22:17:37 INFO LBFGS: Step Size: 33,69
23/03/25 22:17:37 INFO LBFGS: Val and Grad Norm: 0,00712340 (rel: 0,438) 0,00515942
23/03/25 22:17:37 INFO LBFGS: Step Size: 1,500
23/03/25 22:17:37 INFO LBFGS: Val and Grad Norm: 0,00609361 (rel: 0,145) 0,00442296
23/03/25 22:17:37 INFO StrongWolfeLineSearch: Line search t: 0.12429927106991745 fval: 0.005391720278673216 rhs: 0.006093534241055226 cdd: -0.004799561857795583
23/03/25 22:17:37 INFO LBFGS: Step Size: 0,1243
23/03/25 22:17:37 INFO LBFGS: Val and Grad Norm: 0,00539172 (rel: 0,115) 0,00328031
23/03/25 22:17:37 INFO StrongWolfeLineSearch: Line search t: 0.4804681567060297 fval: 0.0046823908398160845 rhs: 0.0053916120304223 cdd: -9.567224902773007E-5
23/03/25 22:17:37 INFO LBFGS: Step Size: 0,4805
23/03/25 22:17:37 INFO LBFGS: Val and Grad Norm: 0,00468239 (rel: 0,132) 0,00141876
23/03/25 22:17:37 INFO LBFGS: Step Size: 1,000
23/03/25 22:17:37 INFO LBFGS: Val and Grad Norm: 0,00456329 (rel: 0,0254) 0,00145630
23/03/25 22:17:38 INFO

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator


lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_349cfaa06043, numClasses = 2, numFeatures = 28
predictions: DataFrame = [V1: double, V2: double ... 31 more fields]

# Evaluation du modèle de détection de fraude

In [24]:
// Afficher les résultats de la prédiction
predictions.select("prediction", "Class", "features").show(5)

val evaluator = new BinaryClassificationEvaluator()
.setLabelCol("Class")
.setRawPredictionCol("rawPrediction")
.setMetricName("areaUnderROC")

val areaUnderROC = evaluator.evaluate(predictions)
println(s"Area under ROC: ${areaUnderROC}")

+----------+-----+--------------------+
|prediction|Class|            features|
+----------+-----+--------------------+
|       0.0|    0|[-28.344757250015...|
|       0.0|    1|[-27.143678422949...|
|       1.0|    0|[-26.619952145208...|
|       0.0|    0|[-23.066841657925...|
|       1.0|    1|[-22.561699259129...|
+----------+-----+--------------------+
only showing top 5 rows

Area under ROC: 0.9705417184464596


evaluator: BinaryClassificationEvaluator = binEval_b0595c9f38c4
areaUnderROC: Double = 0.9705417184464596

# Création d'un RDD pour comparer les prédictions et les vraies valeurs

In [25]:
//Essayez de convertir ces colonnes en Double en utilisant la fonction "cast" de Spark SQL :
import org.apache.spark.sql.functions.col

val predictionAndLabels = predictions.select(col("prediction").cast("Double"), col("Class").cast("Double"))
  .rdd
  .map(row => (row.getDouble(0), row.getDouble(1)))


import org.apache.spark.sql.functions.col


predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[318] at map at cmd24.sc:5

# Transformation des prédictions et labels en RDD de tuples.



In [26]:
val predictionAndLabels = predictions.select("prediction", "Class")
  .withColumn("predictionDouble", $"prediction".cast(DoubleType))
  .withColumn("classDouble", $"Class".cast(DoubleType))
  .rdd
  .map(row => (row.getAs[Double]("predictionDouble"), row.getAs[Double]("classDouble")))


predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[324] at map at cmd25.sc:5

# Evaluation du modèle de détection de fraude


In [27]:
//Le code crée d'abord un RDD predictionAndLabels qui contient des tuples de doubles représentant les prédictions de classe 
//et les étiquettes réelles pour chaque exemple de données. Ensuite,
//il utilise la classe MulticlassMetrics pour créer une instance metrics à partir de ce RDD et calcule la matrice de confusion 
//et les mesures de performance de classification telles que la précision, le rappel et le score F1.
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val predictionAndLabels = predictions.selectExpr("cast(prediction as double)", "cast(Class as double)")
  .rdd
  .map(row => (row.getDouble(0), row.getDouble(1)))

val metrics = new MulticlassMetrics(predictionAndLabels)
println(s"Confusion matrix:\n${metrics.confusionMatrix}")

val precision = metrics.precision(1.0)
val recall = metrics.recall(1.0)
val f1Score = metrics.fMeasure(1.0)

println(s"Precision: $precision")
println(s"Recall: $recall")
println(s"F1-score: $f1Score")


Confusion matrix:
57120.0  13.0  
33.0     67.0  
Precision: 0.8375
Recall: 0.67
F1-score: 0.7444444444444446


import org.apache.spark.mllib.evaluation.MulticlassMetrics


predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[330] at map at cmd26.sc:5
metrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@324825dc
precision: Double = 0.8375
recall: Double = 0.67
f1Score: Double = 0.7444444444444446

# Sélection des meilleures fonctionnalités avec Chi-Square Selector

In [28]:
//jai ajuste le modèle de sélecteur aux données d'entraînement et utilisons ensuite ce modèle pour transformer 
//les données d'entraînement et de test en sélectionnant uniquement les 10 meilleures fonctionnalités
import org.apache.spark.ml.feature.ChiSqSelector

val selector = new ChiSqSelector()
  .setFeaturesCol("features")
  .setLabelCol("Class")
  .setOutputCol("selectedFeatures")
  .setNumTopFeatures(10)

val model = selector.fit(assembledTrainingData)

val selectedTrainingData = model.transform(assembledTrainingData)
val selectedTestData = model.transform(assembledTestData)

23/03/25 22:17:55 ERROR Executor: Exception in task 11.0 in stage 120.0 (TID 913)
org.apache.spark.SparkException: Chi-square test expect factors (categorical values) but found more than 10000 distinct values in column 0.
	at org.apache.spark.mllib.stat.test.ChiSqTest$.$anonfun$chiSquaredFeatures$4(ChiSqTest.scala:111)
	at org.apache.spark.mllib.stat.test.ChiSqTest$.$anonfun$chiSquaredFeatures$4$adapted(ChiSqTest.scala:108)
	at scala.collection.immutable.HashMap$HashMap1.foreach(HashMap.scala:370)
	at scala.collection.immutable.HashMap$HashTrieMap.foreach(HashMap.scala:675)
	at org.apache.spark.mllib.stat.test.ChiSqTest$.$anonfun$chiSquaredFeatures$3(ChiSqTest.scala:108)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.util.collection.ExternalSorter.

: 

# Configuration de l'évaluateur et du validateur

In [29]:
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

// Créer un évaluateur de classification binaire
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("Class")
  .setRawPredictionCol("rawPrediction")
  .setMetricName("areaUnderROC")

import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

// Créer un évaluateur de classification binaire

evaluator: BinaryClassificationEvaluator = binEval_d4e593883d5d

# Grille de recherche des paramètres qui donnent les meilleures performances.

In [30]:
// Créer un objet Grid Search pour explorer différentes combinaisons d'hyperparamètres
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.01, 0.1, 1.0))
  .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
  .addGrid(lr.maxIter, Array(10, 20, 30))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 10,
	logreg_349cfaa06043-regParam: 0.01
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 10,
	logreg_349cfaa06043-regParam: 0.1
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 10,
	logreg_349cfaa06043-regParam: 1.0
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 20,
	logreg_349cfaa06043-regParam: 0.01
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 20,
	logreg_349cfaa06043-regParam: 0.1
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 20,
	logreg_349cfaa06043-regParam: 1.0
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 30,
	logreg_349cfaa06043-regParam: 0.01
},
  {
	logreg_349cfaa06043-elasticNetParam: 0.0,
	logreg_349cfaa06043-maxIter: 30,
...

# Configuration de la validation croisée

In [31]:
// Créer un objet Cross Validator pour évaluer les performances du modèle
val cv = new CrossValidator()
  .setEstimator(lr)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)

cv: CrossValidator = cv_a9795c022163

# Entraînement du modèle de classification de fraude avec CrossValidator

In [32]:
// Entraîner le modèle en utilisant la méthode Cross Validation
val cvModel = cv.fit(assembledTrainingData)

23/03/25 22:18:17 INFO LBFGS: Step Size: 35,75
23/03/25 22:18:17 INFO LBFGS: Val and Grad Norm: 0,0111986 (rel: 0,0864) 0,0139599
23/03/25 22:18:17 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:17 INFO LBFGS: Val and Grad Norm: 0,00739919 (rel: 0,339) 0,00852427
23/03/25 22:18:17 INFO StrongWolfeLineSearch: Line search t: 0.5462093482239323 fval: 0.006034047851508112 rhs: 0.007398941802148653 cdd: -3.810374036991192E-5
23/03/25 22:18:17 INFO LBFGS: Step Size: 0,5462
23/03/25 22:18:17 INFO LBFGS: Val and Grad Norm: 0,00603405 (rel: 0,184) 0,00171503
23/03/25 22:18:17 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:17 INFO LBFGS: Val and Grad Norm: 0,00595802 (rel: 0,0126) 0,00107296
23/03/25 22:18:17 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:17 INFO LBFGS: Val and Grad Norm: 0,00588022 (rel: 0,0131) 0,000744475
23/03/25 22:18:17 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:17 INFO LBFGS: Val and Grad Norm: 0,00583021 (rel: 0,00851) 0,000821299
23/03/25 22:18:17 INFO LBFGS: Step Size: 1,000
2

23/03/25 22:18:23 INFO LBFGS: Step Size: 3,375
23/03/25 22:18:23 INFO LBFGS: Val and Grad Norm: 0,00924275 (rel: 0,000136) 0,000257864
23/03/25 22:18:23 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:23 INFO LBFGS: Val and Grad Norm: 0,00923180 (rel: 0,00118) 0,000610585
23/03/25 22:18:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:24 INFO LBFGS: Val and Grad Norm: 0,00922452 (rel: 0,000789) 0,000286871
23/03/25 22:18:24 INFO StrongWolfeLineSearch: Line search t: 0.42502334915801854 fval: 0.009224227956058281 rhs: 0.009224516673657167 cdd: -1.6008585273502754E-11
23/03/25 22:18:24 INFO LBFGS: Step Size: 0,4250
23/03/25 22:18:24 INFO LBFGS: Val and Grad Norm: 0,00922423 (rel: 3,13e-05) 0,000200115
23/03/25 22:18:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:24 INFO LBFGS: Val and Grad Norm: 0,00922402 (rel: 2,28e-05) 1,30340e-05
23/03/25 22:18:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:24 INFO LBFGS: Val and Grad Norm: 0,00922402 (rel: 1,05e-07) 2,08880e-06
23/03/25 22:18:24 INFO LBFG

23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 0,0319) 8,54653e-05
23/03/25 22:18:29 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 3,07e-07) 6,19126e-07
23/03/25 22:18:29 INFO LBFGS: Step Size: 3,375
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 1,03e-10) 6,53799e-07
23/03/25 22:18:29 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 1,37e-09) 3,20363e-06
23/03/25 22:18:29 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 2,03e-09) 5,46560e-06
23/03/25 22:18:29 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 3,20e-09) 6,30712e-06
23/03/25 22:18:29 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel: 1,69e-09) 5,02178e-06
23/03/25 22:18:29 INFO LBFGS: Step Size: 1,000
23/03/25 22:18:29 INFO LBFGS: Val and Grad Norm: 0,0118675 (rel:

23/03/25 22:18:36 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 3,10e-07) 2,63498e-06
23/03/25 22:18:36 INFO OWLQN: Step Size: 0,5000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 5,25e-09) 4,78164e-06
23/03/25 22:18:36 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 2,17e-08) 2,22533e-06
23/03/25 22:18:36 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 3,33e-09) 1,09056e-06
23/03/25 22:18:36 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 1,27e-09) 5,77038e-07
23/03/25 22:18:36 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 3,68e-10) 3,14677e-07
23/03/25 22:18:36 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:36 INFO OWLQN: Val and Grad Norm: 0,00959760 (rel: 3,06e-10) 2,42348e-07
23/03/25 22:18:36 INFO OWLQN: Step Size: 0,5000
23/03/

23/03/25 22:18:43 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:43 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 1,80e-06) 3,09939e-05
23/03/25 22:18:43 INFO OWLQN: Step Size: 0,5000
23/03/25 22:18:43 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 9,56e-07) 8,04273e-07
23/03/25 22:18:43 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:43 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 2,47e-09) 8,20429e-07
23/03/25 22:18:43 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:43 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 1,02e-09) 1,48985e-06
23/03/25 22:18:43 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:43 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 5,93e-10) 1,69865e-06
23/03/25 22:18:43 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:43 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 2,50e-09) 8,99506e-07
23/03/25 22:18:44 INFO OWLQN: Step Size: 1,000
23/03/25 22:18:44 INFO OWLQN: Val and Grad Norm: 0,0116437 (rel: 6,49e-10) 6,99365e-07
23/03/25 22:18:44 INFO OWLQN: Step Size: 1,000
23/03/25 22:18

...

23/03/25 22:19:23 INFO LBFGS: Val and Grad Norm: 0,0123755 (rel: 8,56e-09) 5,35522e-06
23/03/25 22:19:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:24 INFO LBFGS: Val and Grad Norm: 0,0123755 (rel: 7,99e-10) 2,72026e-06
23/03/25 22:19:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:24 INFO LBFGS: Val and Grad Norm: 0,0123755 (rel: 3,53e-10) 1,41055e-07
23/03/25 22:19:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:24 INFO LBFGS: Val and Grad Norm: 0,0123755 (rel: 2,51e-12) 2,27788e-08
23/03/25 22:19:24 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:24 INFO LBFGS: Val and Grad Norm: 0,0123755 (rel: 5,06e-14) 5,00251e-10
23/03/25 22:19:24 INFO LBFGS: Converged because gradient converged
23/03/25 22:19:24 INFO LBFGS: Step Size: 32,44
23/03/25 22:19:24 INFO LBFGS: Val and Grad Norm: 0,0127550 (rel: 0,00734) 0,0154403
23/03/25 22:19:25 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:25 INFO LBFGS: Val and Grad Norm: 0,00847962 (rel: 0,335) 0,00992973
23/03/25 22:19:25 INFO StrongWolfeLineSearch: Li

23/03/25 22:19:29 INFO OWLQN: Step Size: 1,000
23/03/25 22:19:29 INFO OWLQN: Val and Grad Norm: 0,00998919 (rel: 0,00175) 0,000444854
23/03/25 22:19:29 INFO OWLQN: Step Size: 1,000
23/03/25 22:19:29 INFO OWLQN: Val and Grad Norm: 0,00998257 (rel: 0,000663) 0,000558930
23/03/25 22:19:29 INFO OWLQN: Step Size: 1,000
23/03/25 22:19:29 INFO OWLQN: Val and Grad Norm: 0,00997480 (rel: 0,000778) 0,00120140
23/03/25 22:19:29 INFO OWLQN: Step Size: 1,000
23/03/25 22:19:29 INFO OWLQN: Val and Grad Norm: 0,00996505 (rel: 0,000977) 0,000683176
23/03/25 22:19:29 INFO OWLQN: Converged because max iterations reached
23/03/25 22:19:30 INFO OWLQN: Converged because gradient converged
23/03/25 22:19:30 INFO OWLQN: Converged because gradient converged
23/03/25 22:19:31 INFO OWLQN: Step Size: 7,153
23/03/25 22:19:31 INFO OWLQN: Val and Grad Norm: 0,0112666 (rel: 0,123) 0,0132053
23/03/25 22:19:31 INFO OWLQN: Step Size: 0,2500
23/03/25 22:19:31 INFO OWLQN: Val and Grad Norm: 0,0106281 (rel: 0,0567) 0,00893

23/03/25 22:19:37 INFO OWLQN: Step Size: 1,000
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119646 (rel: 0,00397) 0,000242946
23/03/25 22:19:37 INFO OWLQN: Step Size: 0,5000
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119635 (rel: 9,24e-05) 0,000517663
23/03/25 22:19:37 INFO OWLQN: Step Size: 0,2500
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119621 (rel: 0,000124) 0,000221082
23/03/25 22:19:37 INFO OWLQN: Step Size: 1,000
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119608 (rel: 0,000103) 0,000641184
23/03/25 22:19:37 INFO OWLQN: Step Size: 0,2500
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119581 (rel: 0,000225) 0,000232653
23/03/25 22:19:37 INFO OWLQN: Step Size: 0,2500
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119573 (rel: 6,71e-05) 0,000256217
23/03/25 22:19:37 INFO OWLQN: Step Size: 0,5000
23/03/25 22:19:37 INFO OWLQN: Val and Grad Norm: 0,0119571 (rel: 1,56e-05) 0,000327052
23/03/25 22:19:37 INFO OWLQN: Converged because max iterat

23/03/25 22:19:45 INFO OWLQN: Converged because gradient converged
23/03/25 22:19:46 INFO LBFGS: Step Size: 33,69
23/03/25 22:19:46 INFO LBFGS: Val and Grad Norm: 0,0121234 (rel: 0,0441) 0,0146903
23/03/25 22:19:46 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:46 INFO LBFGS: Val and Grad Norm: 0,00809824 (rel: 0,332) 0,00940299
23/03/25 22:19:46 INFO StrongWolfeLineSearch: Line search t: 0.503524486923693 fval: 0.006397182048828225 rhs: 0.008097958267623391 cdd: -6.167367411543588E-4
23/03/25 22:19:46 INFO LBFGS: Step Size: 0,5035
23/03/25 22:19:46 INFO LBFGS: Val and Grad Norm: 0,00639718 (rel: 0,210) 0,00190423
23/03/25 22:19:46 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:46 INFO LBFGS: Val and Grad Norm: 0,00630011 (rel: 0,0152) 0,00151576
23/03/25 22:19:46 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:46 INFO LBFGS: Val and Grad Norm: 0,00624964 (rel: 0,00801) 0,000824868
23/03/25 22:19:46 INFO LBFGS: Step Size: 1,000
23/03/25 22:19:46 INFO LBFGS: Val and Grad Norm: 0,00621970 (rel: 0,

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_a9795c022163

# Utilisation du modèle pour prédire la fraude

In [33]:
// Utiliser le modèle pour prédire les classes de l'ensemble de test
val predictions = cvModel.transform(assembledTestData)

predictions: DataFrame = [V1: double, V2: double ... 31 more fields]

# Calcul de l'aire sous la courbe ROC (Receiver Operating Characteristic) pour évaluer la performance du modèle et affichage de la valeur obtenue.

In [48]:
// Évaluer les performances du modèle en utilisant l'évaluateur de classification binaire
val areaUnderROC = evaluator.evaluate(predictions)
println(s"Zone sous le ROC: $areaUnderROC")

Zone sous le ROC: 0.9752201004673505


areaUnderROC: Double = 0.9752201004673505

# Définition du chemin du modèle

In [45]:
val modelPath = "/model"

modelPath: String = "/model"

# Sauvegarde du modèle entraîné au chemin modelPath.

In [46]:
model.save(modelPath)

: 